In [1]:
### https://ml-challenge.mercadolibre.com/

import pandas as pd
import numpy as np
import gc

# Convert 64bit to 32bit
def reduce_mem(df):
    for col in df.columns:
        if df[col].dtype == 'int64':
            df[col] = df[col].astype('int32')
        if df[col].dtype == 'float64':
            df[col] = df[col].astype('float32')
    _ = gc.collect()
    return df

In [2]:
%%time

train = pd.read_parquet('../input/train_data.parquet')
test = pd.read_csv('../input/test_data.csv')
train = train.merge(test[['sku','target_stock']], on='sku', how='left' )

train['date'] = pd.to_datetime(train['date'])
train['month'] = train['date'].dt.month
train['dow'] = train['date'].dt.dayofweek

file1 = open('../input/items_static_metadata_full.jl', 'r')
lines = file1.readlines()
for i in range(len(lines)):
    lines[i] = eval( lines[i].replace('null', '-1') )
items = pd.DataFrame().from_dict(lines)

train = train.merge(items, on='sku', how='left' )

train['item_title'] = train['item_title'].apply(lambda x: x.lower() )
train['item_title_id'] = train['item_title'].factorize()[0]
train['item_title_0'] = train['item_title'].apply(lambda x: ' '.join(x.split())[:20] ).factorize()[0]
train['item_title_1'] = train['item_title'].apply(lambda x: ' '.join(x.split())[-20:] ).factorize()[0]
del train['item_title']

for col in ['currency', 'listing_type', 'shipping_logistic_type', 'shipping_payment', 'item_domain_id', 'item_title_id' ,'item_id', 'site_id', 'product_id', 'product_family_id']:
    train[col] = train[col].factorize()[0]
    
print(train.tail())

             sku       date  sold_quantity  current_price  currency  \
37660274  129187 2021-03-31              0        22057.0         3   
37660275    6707 2021-03-31              0        26999.0         3   
37660276  170355 2021-03-31              0         3400.0         3   
37660277  246568 2021-03-31              0         6289.0         3   
37660278   49718 2021-03-31              0         1990.0         3   

          listing_type  shipping_logistic_type  shipping_payment  \
37660274             0                       2                 0   
37660275             0                       1                 0   
37660276             0                       2                 1   
37660277             0                       0                 0   
37660278             0                       2                 1   

          minutes_active  target_stock  month  dow  item_domain_id  item_id  \
37660274      267.710767           8.0      3    2            8408   517891   
376602

In [3]:
%%time

# Sort by date
train = train.sort_values(['sku','date'])

# Convert to 32bit
train = reduce_mem(train)

# Write to disk
train.to_parquet('train.parquet')
print('Done')

Done
CPU times: user 33.9 s, sys: 8.38 s, total: 42.3 s
Wall time: 41.7 s
